# Train Custom Datasets

In [ ]:
"""
mmaction2をimportできるようにPATHを通しておく
"""
import sys
from pathlib import Path

current_dir = Path().resolve()
root_dir = current_dir.parents[1]
mmaction2_dir = root_dir.joinpath('mmaction2')

if not (f'{root_dir}' in sys.path and f'{mmaction2_dir}' in sys.path):
    sys.path.append(f'{root_dir}')
    sys.path.append(f'{mmaction2_dir}')
    sys.path
else:
    print(f'[{root_dir}] and [{mmaction2_dir}] directories are alreardy registerd.')

In [ ]:
"""
mmaction2をclone
(Apple silicon使用の場合は、dtypeを変更する)
"""
if not mmaction2_dir.is_dir():
    # Install mmaction2
    %cd {root_dir}
    !git clone https://github.com/open-mmlab/mmaction2.git -b main
    %pip install -U openmim
    !mim install mmengine
    !mim install mmcv
    !mim install mmdet

    # Cannot convert a MPS Tensor to float64 dtype as the MPS framework doesn't support float64. Please use float32 instead.
    # のエラー対応
    import torch
    if torch.backends.mps.is_available():
        !sed -i "" s/"a, device=cls_scores.device)"/"a, device=cls_scores.device, dtype=torch.float32)"/g {mmaction2_dir}/mmaction/models/heads/base.py
    %cd {current_dir}
else:
    print(f'[{mmaction2_dir}] directory is already exists.')

In [ ]:
# Check Pytorch installation
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

# Check MMAction2 installation
from mmaction2 import mmaction
print(mmaction.__version__)

# Check MMCV installation
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print(get_compiling_cuda_version())
print(get_compiler_version())

# Check MMEngine installation
from mmengine.utils.dl_utils import collect_env
print(collect_env())

In [ ]:
# After downloading the data, we need to check the annotation format
!cat data/toilet/toilet_train_video.txt

### Modify the config

In [ ]:
from mmaction2.mmaction.apis import inference_recognizer, init_recognizer
from mmengine import Config
from mmengine.runner import set_random_seed


cfg = Config.fromfile('./config/toilet_config.py')

# We can use the pre-trained TSN model
cfg.load_from = None
# Set up working dir to save files and logs.
cfg.work_dir = 'checkpoint/toilet'

# The original learning rate (LR) is set for 8-GPU training.
# We divide it by 8 since we only use one GPU.
cfg.train_dataloader.batch_size = cfg.train_dataloader.batch_size // 16
cfg.val_dataloader.batch_size = cfg.val_dataloader.batch_size // 16
cfg.optim_wrapper.optimizer.lr = cfg.optim_wrapper.optimizer.lr / 8 / 16
cfg.train_cfg.max_epochs = 10

cfg.train_dataloader.num_workers = 2
cfg.val_dataloader.num_workers = 2
cfg.test_dataloader.num_workers = 2

# We can initialize the logger for training and have a look
# at the final config used for training
print(f'Config:\n{cfg.pretty_text}')


### Train a new recognizer

Finally, lets initialize the dataset and recognizer, then train a new recognizer!

In [ ]:
import os
import os.path as osp
import torch
import mmengine
from mmengine.runner import Runner


# Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend
# 上記エラーの回避
os.environ['OMP_NUM_THREADS'] = '1'

if torch.backends.mps.is_available():
    torch.device('mps')

# Create work_dir
mmengine.mkdir_or_exist(osp.abspath(cfg.work_dir))

# build the runner from config
runner = Runner.from_cfg(cfg)

# start training
runner.train()

## Test the trained recognizer

After finetuning the recognizer, let's check the prediction results!

In [ ]:
runner.test()

In [ ]:
%cd {root_dir}
!PYTHONPATH=mmaction2 python mmaction2/demo/demo.py --device mps camenashi_kun/classification/config/toilet_config.py \
    camenashi_kun/classification/checkpoint/toilet/epoch_9.pth \
    camenashi_kun/classification/data/toilet/val/20240213-105939.mp4 camenashi_kun/classification/data/toilet/label_map.txt